# Sex differences in lung adenocarcinoma (LUAD)
Authors: Mia Shapoval<sup>1</sup>, Camila Lopes-Ramos<sup>2,3</sup>, Marouen Ben Guebila<sup>2</sup>, John Quackenbsuh<sup>2,3</sup>

<sup>1</sup>Boston University Academy, Boston, MA.

<sup>2</sup>Department of Biostatistics, Harvard T.H. Chan School of Public Health, Boston, MA.

<sup>3</sup>Channing Division of Network Medicine, Brigham and Women's Hospital and Harvard Medical School, Boston, MA.

## 1. Introduction 

It’s known that males and females exhibit diseases differently. However, there’s still very little research on why this is true<sup>1</sup>. Most studies about diseases or the effects of treatments don’t take sex and gender into account, if they include women at all. In order to study the difference between males and females we need to look at the processes that regulate gene expression.

This analysis aims to identify sex differences in lung adenocarcinoma (LUAD). This will be done by first reconstructing gene regulatory networks for LUAD and normal lung by using the PANDA<sup>2</sup> method. The PANDA method (Passing Attributes between Networks for Data Assimilation), estimates bipartite graphs where transcription factors (TFs) and their target genes are the nodes, and edge weights show the probability the TF regulates the corresponding target gene. Next, differential targeting between males and females are computed for LUAD and normal lung. This is to find gene regulatory differences between the male and female LUAD networks, and the male and female normal lung networks. Finally, a gene set enrichment analysis for LUAD and normal lung is completed. A gene set enrichment analysis identifies the biological processes enriched for genes highly targeted in males and those processes enriched for genes highly targeted in females. 

The LUAD data used is provided from TCGA<sup>3</sup>, a NIH project that has gathered gene expression data from various cancer types. The normal lung data is from GTEx<sup>4</sup>. 

### Loading libraries 

The following libraries are used in this analysis:

In [ ]:
library(netZooR)
library(visNetwork) #for network visualization 
library(fgsea) #for gene set enrichment analysis 
library(ggplot2) #for bubble plot

Then we will set the project path to load input data and save results.

In [ ]:
ppath = '/opt/data/netZooR/lungcancer/'

Here we will load a table with gene Ensembl ID and HGNC from GENCODE V22. We will use this to convert Ensembl IDs to HGNC, required for gene set enrichment analysis.

In [ ]:
annot <- read.delim(paste0(ppath,"gencode_annotation_v22_complete.txt"), stringsAsFactors=F)

In [ ]:
head(annot)

## 2. Reconstruction of LUAD differential gene regulatory networks in males and females

### 2.1. Reconstruction of gene regulatory network in healthy lung

#### 2.1.1. Reconstruction of gene regulatory network in females


First, let's start by specifying the path to the input data. Since the analysis may take awhile, we will use the precomputed results by setting the `precomputed` argument to 1.

In [ ]:
precomputed <- 1

In [ ]:
if (precomputed == 0){
    inputargs <- c("lung/luad/data",
                   paste0(ppath,"exp_logQsmooth_female_lung.txt"),
                   paste0(ppath,"motif_640_30243_female_ZeroYgenesPrior.txt"),
                   paste0(ppath,"ppi_640.txt"),
                   "female_lung")

    inputargs <- commandArgs(TRUE)
    print(inputargs)

    data_dir <- inputargs[1]
    exp_file <- inputargs[2]
    motif_file <- inputargs[3]
    ppi_file  <- inputargs[4]
    tag <- inputargs[5]
}

Next, we will load the gene experession, ppi, and motif data.

In [ ]:
if (precomputed == 0){
    exp <- read.delim(paste0(ppath,"exp_logQsmooth_female_lung.txt"), check.names = FALSE)
    ppi <- read.delim(paste0(ppath,"ppi_640.txt"), stringsAsFactors=F, header=F)
    motif <- read.delim(paste0(ppath,"motif_640_30243_female_ZeroYgenesPrior.txt"), stringsAsFactors=F, header=F)
}

Now, we will build the gene regulatory network by using the PANDA method<sup>1</sup> by integrating three sources of data. We will use the `intersection` mode which takes the intersection genes between motif and gene co-expression networks, and the intersecting TFs between the motif and ppi networks.

In [ ]:
if (precomputed == 0){
    panda_results <- panda(motif,exp, ppi, mode="intersection")
    panda <- panda_results@regNet
    out_file <- paste0("../panda/panda_",tag,".rdata")
    save(panda, file=out_file)
} else if (precomputed == 1) {
    load(paste0(ppath,"panda_female_lung.rdata"))
    panda_female_lung <- panda
}

Convert the gene IDs from ensembl to symbol format 

In [ ]:
a <- colnames(panda_female_lung)
geneId <- annot[match(a,annot$gene_id),"gene_name"]
colnames(panda_female_lung) <-  geneId

#### 2.1.2. Reconstruction of gene regulatory network in males

In [ ]:
if (precomputed == 0){
    inputargs <- c("lung/luad/data",
                   paste0(ppath,"exp_logQsmooth_male_lung.txt"),
                   paste0(ppath,"motif_640_30243_male.txt"),
                   paste0(ppath,"ppi_640.txt"),
                   "male_lung")

    inputargs <- commandArgs(TRUE)
    print(inputargs)

    data_dir <- inputargs[1]
    exp_file <- inputargs[2]
    motif_file <- inputargs[3]
    ppi_file  <- inputargs[4]
    tag <- inputargs[5]
}

Again, we start by loading the input data.

In [ ]:
if (precomputed == 0){
    exp <- read.delim(paste0(ppath,"exp_logQsmooth_male_lung.txt"), check.names = FALSE)
    ppi <- read.delim(paste0(ppath,"ppi_640.txt"), stringsAsFactors=F, header=F)
    motif <- read.delim(paste0(ppath,"motif_640_30243_male.txt"), stringsAsFactors=F, header=F)
}

Now, we run the PANDA method using the `intersection` mode.

In [ ]:
if (precomputed == 0){
    panda_results <- panda(motif,exp, ppi, mode="intersection")
    panda <- panda_results@regNet
    out_file <- paste0("../panda/panda_",tag,".rdata")
    save(panda, file=out_file)
} else if (precomputed == 1) {
    load(paste0(ppath,"panda_male_lung.rdata"))
    panda_male_lung <- panda
}

Convert the gene IDs from ensembl to symbol format 

In [ ]:
a <- colnames(panda_male_lung)
geneId <- annot[match(a,annot$gene_id),"gene_name"]
colnames(panda_male_lung) <-  geneId

### 2.2. Reconstruction of gene regulatory network in LUAD

#### 2.2.1. Reconstruction of gene regulatory network in females

In [ ]:
if (precomputed == 0){
    inputargs <- c("lung/luad/data",
                   paste0(ppath,"exp_logQsmooth_female_luad.txt"),
                   paste0(ppath,"motif_640_30243_female_ZeroYgenesPrior.txt"),
                   paste0(ppath,"ppi_640.txt"),
                   "female_lung")

    inputargs <- commandArgs(TRUE)
    print(inputargs)

    data_dir <- inputargs[1]
    exp_file <- inputargs[2]
    motif_file <- inputargs[3]
    ppi_file  <- inputargs[4]
    tag <- inputargs[5]
} 

First, we need to load the input data.

In [ ]:
if (precomputed == 0){
    exp <- read.delim(paste0(ppath,"exp_logQsmooth_female_luad.txt"), check.names = FALSE)
    ppi <- read.delim(paste0(ppath,"ppi_640.txt"), stringsAsFactors=F, header=F)
    motif <- read.delim(paste0(ppath,"motif_640_30243_female_ZeroYgenesPrior.txt"), stringsAsFactors=F, header=F)
}

Then, we run the PANDA method using the `intersection` mode.

In [ ]:
if (precomputed == 0){
    panda_results <- panda(motif,exp, ppi, mode="intersection")
    panda <- panda_results@regNet
    out_file <- paste0("../panda/panda_",tag,".rdata")
    save(panda, file=out_file)
} else if (precomputed == 1) {
    load(paste0(ppath,"panda_female_luad.rdata"))
    panda_female_luad <- panda
}

Convert the gene IDs from ensembl to symbol format 

In [ ]:
a <- colnames(panda_female_luad)
geneId <- annot[match(a,annot$gene_id),"gene_name"]
colnames(panda_female_luad) <-  geneId

#### 2.2.1. Reconstruction of gene regulatory network in males

In [ ]:
if (precomputed == 0){
    inputargs <- c("lung/luad/data",
                   paste0(ppath,"exp_logQsmooth_male_luad.txt"),
                   paste0(ppath,"motif_640_30243_male.txt"),
                   paste0(ppath,"ppi_640.txt"),
                   "male_lung")

    inputargs <- commandArgs(TRUE)
    print(inputargs)

    data_dir <- inputargs[1]
    exp_file <- inputargs[2]
    motif_file <- inputargs[3]
    ppi_file  <- inputargs[4]
    tag <- inputargs[5]
}

Again, we load the input data.

In [ ]:
if (precomputed == 0){
    exp <- read.delim(paste0(ppath,"exp_logQsmooth_male_luad.txt"), check.names = FALSE)
    ppi <- read.delim(paste0(ppath,"ppi_640.txt"), stringsAsFactors=F, header=F)
    motif <- read.delim(paste0(ppath,"motif_640_30243_male.txt"), stringsAsFactors=F, header=F)
} 

Finally, we run the PANDA method using the `intersection` mode.

In [ ]:
if (precomputed == 0){
    panda_results <- panda(motif,exp, ppi, mode="intersection")
    panda <- panda_results@regNet
    out_file <- paste0("../panda/panda_",tag,".rdata")
    save(panda, file=out_file)
} else if (precomputed == 1) {
    load(paste0(ppath,"panda_male_luad.rdata"))
    panda_male_luad <- panda
}

Convert the gene IDs from ensembl to symbol format 

In [ ]:
a <- colnames(panda_male_luad)
geneId <- annot[match(a,annot$gene_id),"gene_name"]
colnames(panda_male_luad) <-  geneId

## 3. Visualizing differential networks between normal lung and LUAD
### 3.1. Visualizing differential networks between female normal lung and female LUAD

Since PANDA produces genome scale networks, it's hard to visualize all the edges. Therefore, we will select the most important edges.

In [ ]:
num_edges <- 100

In [ ]:
edges = matrix(0L, num_edges, 3)
colnames(edges) = c("from","to","value")
edges = as.data.frame(edges)

In [ ]:
diffNet = panda_female_luad - panda_female_lung

In [ ]:
sort_mat = order(as.matrix(abs(diffNet)), decreasing = TRUE)
edges$value  = as.matrix(diffNet)[sort_mat[1:num_edges]]

In [ ]:
geneIdsTop = (sort_mat[1:num_edges] %/% dim(diffNet)[1]) + 1
tfIdsTop = sort_mat[1:num_edges] %% dim(diffNet)[1]
nTFs = dim(diffNet)[1]
tfIdsTop[tfIdsTop == 0] = nTFs

edges$to = colnames(diffNet)[geneIdsTop]
edges$from = rownames(diffNet)[tfIdsTop]
edges$arrows = "to"
edges$color = ifelse(edges$value > 0, "green", "red")
edges$value = abs(edges$value)

nodes = data.frame(id = unique(as.vector(as.matrix(edges[,c(1,2)]))), label=unique(as.vector(as.matrix(edges[,c(1,2)]))))
nodes$group = ifelse(nodes$id %in% edges$from, "TF", "gene")
#nodes$value = c(unique(diffTF[tfIdsTop]), unique(diffGene[geneIdsTop]))

After setting all the parameters for visulaization, we will color the edges in green if they are larger in LUAD and in red if they are larger in normal tissue.

In [ ]:
net <- visNetwork(nodes, edges, width = "100%")
net <- visGroups(net, groupname = "TF", shape = "triangle",
                 color = list(background = "orange", border="black"))
net <- visGroups(net, groupname = "gene", shape = "dot",       
                 color = list(background = "darkblue", border="black"))
visLegend(net, main="Legend", position="right", ncol=1)

### 3.2. Visualizing differential networks between male normal lung and male LUAD

In [ ]:
num_edges <- 100

In [ ]:
edges = matrix(0L, num_edges, 3)
colnames(edges) = c("from","to","value")
edges = as.data.frame(edges)

In [ ]:
diffNet = panda_male_luad - panda_male_lung

In [ ]:
sort_mat = order(as.matrix(abs(diffNet)), decreasing = TRUE)
edges$value  = as.matrix(diffNet)[sort_mat[1:num_edges]]

In [ ]:
geneIdsTop = (sort_mat[1:num_edges] %/% dim(diffNet)[1]) + 1
tfIdsTop = sort_mat[1:num_edges] %% dim(diffNet)[1]
nTFs = dim(diffNet)[1]
tfIdsTop[tfIdsTop == 0] = nTFs

edges$to = colnames(diffNet)[geneIdsTop]
edges$from = rownames(diffNet)[tfIdsTop]
edges$arrows = "to"
edges$color = ifelse(edges$value > 0, "green", "red")
edges$value = abs(edges$value)

nodes = data.frame(id = unique(as.vector(as.matrix(edges[,c(1,2)]))), label=unique(as.vector(as.matrix(edges[,c(1,2)]))))
nodes$group = ifelse(nodes$id %in% edges$from, "TF", "gene")
#nodes$value = c(unique(diffTF[tfIdsTop]), unique(diffGene[geneIdsTop]))

In [ ]:
net <- visNetwork(nodes, edges, width = "100%")
net <- visGroups(net, groupname = "TF", shape = "triangle",
                 color = list(background = "orange", border="black"))
net <- visGroups(net, groupname = "gene", shape = "dot",       
                 color = list(background = "darkblue", border="black"))
visLegend(net, main="Legend", position="right", ncol=1)

## 4. Differential analysis between normal lung and LUAD
### 4.1. Computing differential targeting between normal lung and LUAD

In [ ]:
tftar_female_lung <- rowSums(panda_female_lung)
genetar_female_lung <- colSums(panda_female_lung)

tftar_female_luad <- rowSums(panda_female_luad)
genetar_female_luad <- colSums(panda_female_luad)

tftar_male_lung <- rowSums(panda_male_lung)
genetar_male_lung <- colSums(panda_male_lung)

tftar_male_luad <- rowSums(panda_male_luad)
genetar_male_luad <- colSums(panda_male_luad)

First, we will compute TF targeting and gene targeting on the healthy lung.

In [ ]:
tfttar_dif_lung <- tftar_female_lung - tftar_male_lung
genetar_dif_lung <- genetar_female_lung - genetar_male_lung

Next, we will do the same on the LUAD lung.

In [ ]:
tfttar_dif_luad <- tftar_female_luad - tftar_male_luad
genetar_dif_luad <- genetar_female_luad - genetar_male_luad

### 4.2. Gene set enrichment analysis for LUAD

In this section, we will analyze the list of top-targeted genes in a library of biological processes using Gene Set Enrichment Analysis (GSEA).

In [ ]:
pathwayfile <- "/opt/data/netZooR/lungcancer/c2.cp.kegg.v7.1.symbols.gmt"
pathways <- gmtPathways(pathwayfile)

In [ ]:
fgseaRes <- fgsea(pathways, genetar_dif_luad, minSize=15, maxSize=500, nperm=10000)

GSEA shows an error because there were duplicate gene names in our list. This is due to the ENSG to Symbol conversion that can map several ENSEMBL IDs to the same gene symbol. We will keep the duplicates for now, however a better approach consists of using a GSEA package that accepts ENSEMBL gene ids.

In [ ]:
head(fgseaRes)
sig <- fgseaRes[fgseaRes$padj < 0.05,]

The following command lines allows us to take the top 10 KEGG terms in female LUAD.

In [ ]:
sig$pathway[sig$NES > 0][1:10]

By setting the normalized enrichment score (NES) to a negative value, we obtain the top 10 enriched terms in male LUAD.

In [ ]:
sig$pathway[sig$NES < 0][1:10]

In [ ]:
dat <- data.frame(fgseaRes)
# Settings
fdrcut <- 0.05 # FDR cut-off to use as output for significant signatures
dencol_neg <- "blue" # bubble plot color for negative ES
dencol_pos <- "red" # bubble plot color for positive ES
signnamelength <- 4 # set to remove prefix from signature names (2 for "GO", 4 for "KEGG", 8 for "REACTOME")
asp <- 3 # aspect ratio of bubble plot
charcut <- 100 # cut signature name in heatmap to this nr of characters
# Make signature names more readable
a <- as.character(dat$pathway) # 'a' is a great variable name to substitute row names with something more readable
for (j in 1:length(a)){
  a[j] <- substr(a[j], signnamelength+2, nchar(a[j]))
}
a <- tolower(a) # convert to lower case (you may want to comment this out, it really depends on what signatures you are looking at, c6 signatures contain gene names, and converting those to lower case may be confusing)
for (j in 1:length(a)){
  if(nchar(a[j])>charcut) { a[j] <- paste(substr(a[j], 1, charcut), "...", sep=" ")}
} # cut signature names that have more characters than charcut, and add "..."
a <- gsub("_", " ", a)
dat$NAME <- a
# Determine what signatures to plot (based on FDR cut)
dat2 <- dat[dat[,"padj"]<fdrcut,]
dat2 <- dat2[order(dat2[,"padj"]),] 
dat2$signature <- factor(dat2$NAME, rev(as.character(dat2$NAME)))
# Determine what labels to color
sign_neg <- which(dat2[,"NES"]<0)
sign_pos <- which(dat2[,"NES"]>0)
# Color labels
signcol <- rep(NA, length(dat2$signature))
signcol[sign_neg] <- dencol_neg # text color of negative signatures
signcol[sign_pos] <- dencol_pos # text color of positive signatures
signcol <- rev(signcol) # need to revert vector of colors, because ggplot starts plotting these from below
# Plot bubble plot
g<-ggplot(dat2, aes(x=padj,y=signature,size=size))
g+geom_point(aes(fill=NES), shape=21, colour="white")+
  theme_bw()+ # white background, needs to be placed before the "signcol" line
  xlim(0,fdrcut)+
  scale_size_area(max_size=10,guide="none")+
  scale_fill_gradient2(low=dencol_neg, high=dencol_pos)+
  theme(axis.text.y = element_text(colour=signcol))+
  theme(aspect.ratio=asp, axis.title.y=element_blank()) # test aspect.ratio

### 4.3. Gene set enrichment analysis for normal lung

In [ ]:
pathways <- gmtPathways(pathwayfile)

In [ ]:
fgseaRes <- fgsea(pathways, genetar_dif_lung, minSize=15, maxSize=500, nperm=10000)

In [ ]:
head(fgseaRes)
sig <- fgseaRes[fgseaRes$padj < 0.05,]

We again use the following command lines allows us to take the top 10 KEGG terms in female normal lung.

In [ ]:
sig$pathway[sig$NES > 0][1:10]

Just like before, we set the normalized enrichment score (NES) to a negative value, we obtain the top 10 enriched terms in male normal lung.

In [ ]:
sig$pathway[sig$NES < 0][1:10]

In [ ]:
dat <- data.frame(fgseaRes)
# Settings
fdrcut <- 0.05 # FDR cut-off to use as output for significant signatures
dencol_neg <- "blue" # bubble plot color for negative ES
dencol_pos <- "red" # bubble plot color for positive ES
signnamelength <- 4 # set to remove prefix from signature names (2 for "GO", 4 for "KEGG", 8 for "REACTOME")
asp <- 3 # aspect ratio of bubble plot
charcut <- 100 # cut signature name in heatmap to this nr of characters
# Make signature names more readable
a <- as.character(dat$pathway) # 'a' is a great variable name to substitute row names with something more readable
for (j in 1:length(a)){
  a[j] <- substr(a[j], signnamelength+2, nchar(a[j]))
}
a <- tolower(a) # convert to lower case (you may want to comment this out, it really depends on what signatures you are looking at, c6 signatures contain gene names, and converting those to lower case may be confusing)
for (j in 1:length(a)){
  if(nchar(a[j])>charcut) { a[j] <- paste(substr(a[j], 1, charcut), "...", sep=" ")}
} # cut signature names that have more characters than charcut, and add "..."
a <- gsub("_", " ", a)
dat$NAME <- a
# Determine what signatures to plot (based on FDR cut)
dat2 <- dat[dat[,"padj"]<fdrcut,]
dat2 <- dat2[order(dat2[,"padj"]),] 
dat2$signature <- factor(dat2$NAME, rev(as.character(dat2$NAME)))
# Determine what labels to color
sign_neg <- which(dat2[,"NES"]<0)
sign_pos <- which(dat2[,"NES"]>0)
# Color labels
signcol <- rep(NA, length(dat2$signature))
signcol[sign_neg] <- dencol_neg # text color of negative signatures
signcol[sign_pos] <- dencol_pos # text color of positive signatures
signcol <- rev(signcol) # need to revert vector of colors, because ggplot starts plotting these from below
# Plot bubble plot
g<-ggplot(dat2, aes(x=padj,y=signature,size=size))
g+geom_point(aes(fill=NES), shape=21, colour="white")+
  theme_bw()+ # white background, needs to be placed before the "signcol" line
  xlim(0,fdrcut)+
  scale_size_area(max_size=10,guide="none")+
  scale_fill_gradient2(low=dencol_neg, high=dencol_pos)+
  theme(axis.text.y = element_text(colour=signcol))+
  theme(aspect.ratio=asp, axis.title.y=element_blank()) # test aspect.ratio

# 5. Conclusion 

This network analysis identified biological processes in LUAD that are differentially targeted between males and females, such as pathways in cancer, and in immune processes. There were also many drug metabolism-linked genes identified as differentially targeted between males and females. This could support previous work showing that men and women respond differently to drugs.<sup>5</sup>  

In comparing males and females normal lung, some of the differences were linked to immuno-processes. This means that there are immuno-differences in males and females, which could be linked to the fact women are more likely to get lung cancer than males.<sup>6</sup>


This analysis was able to identify sex differences in LUAD by completing a differential targeting and a gene enrichment analysis. The data produced from this analysis will hopefully aid in future research into sex differences in diseases, and provide others with more information about LUAD.




# References 

1- Lopes-Ramos, Camila M., John Quackenbush, and Dawn L. DeMeo. "Genome-wide sex and gender differences in cancer." Frontiers in Oncology 10 (2020): 2486

2- Glass, Kimberly, et al. "Passing messages between biological networks to refine predicted interactions." PloS one 8.5 (2013): e64832.

3- Tomczak, Katarzyna, Patrycja Czerwińska, and Maciej Wiznerowicz. "The Cancer Genome Atlas (TCGA): an immeasurable source of knowledge." Contemporary oncology 19.1A (2015): A68.

4- GTEx Consortium. "The Genotype-Tissue Expression (GTEx) pilot analysis: Multitissue gene regulation in humans." Science 348.6235 (2015): 648-660.

5- Lopes-Ramos CM, Kuijjer ML, Ogino S, Fuchs CS, DeMeo DL, Glass K, Quackenbush J. Gene Regulatory Network Analysis Identifies Sex-Linked Differences in Colon Cancer Drug Metabolism. Cancer Res. 2018 Oct 1;78(19):5538-5547.

6- Lopes-Ramos, Camila M., et al. "Sex differences in gene expression and regulatory networks across 29 human tissues." Cell reports 31.12 (2020): 107795.
